In [1]:
!pip install -qU langchain_community pypdf
!pip install langchain-groq
!pip install sentence_transformers
!pip install faiss-gpu
!pip install langgraph
!pip install --upgrade pydantic

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-cloud-bigquery 2.34.4 requires packaging<22.0dev,>=14.3, but you have packaging 24.2 which is incompatible.
jupyterlab 4.3.1 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
jupyterlab-lsp 5.1.0 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
kfp 2.5.0 requires google-cloud-storage<3,>=2.2.1, but you have google-cloud-storage 1.44.0 which is incompatible.
kfp 2.5.0 requires requests-toolbelt<1,>=0.8.0, but you have requests-toolbelt 1.0.0 which is incompatible.
libpysal 4.9.2 requires shapely>=2.0.1, but you have shapely 1.8.5.post1 which is incompatible.
mlxtend 0.23.3 requires scikit-learn>=1.3.1, but you have scikit-learn 1.2.2 which is incompatible.
plotnine 0.14.3 requires matplotlib>=3.8.0, but you have matplotlib 3.7.5 which is inco

In [2]:
# LIBRARY
from langchain_community.document_loaders import PyPDFLoader, Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import MessagesPlaceholder
from langchain.chains import create_history_aware_retriever

from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from typing import List
from operator import itemgetter
from langchain_groq import ChatGroq
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
import os

In [3]:
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_PROJECT'] = 'advanced-rag'
os.environ['LANGCHAIN_API_KEY'] = "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"
os.environ['GROQ_API_KEY'] = "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"
os.environ['TAVILY_API_KEY'] = "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"

In [4]:
# UPLOAD THE DOCUMENTS
documents = PyPDFLoader('/kaggle/input/kitchen-dataset/cook_book.pdf')
documents = documents.load()

In [5]:
documents

[Document(metadata={'source': '/kaggle/input/kitchen-dataset/cook_book.pdf', 'page': 0}, page_content='1. Nargisy Kababs: \n Ingredients \n\uf0b7 1 kg minced meat \n\uf0b7 2 medium onions, cut into large pieces \n\uf0b7 1/2 cup curd \n\uf0b7 1 tbsp ginger-garlic paste \n\uf0b7 1 tsp red chilli powder \n\uf0b7 1 tsp garam masala \n\uf0b7 1/4 tsp turmeric powder \n\uf0b7 1 tbsp soy sauce \n\uf0b7 salt as per taste \n\uf0b7 1 tbsp vinegar \n\uf0b7 1/4 cup \nFor Filling: \n\uf0b7 4 boiled eggs, finely chopped \n\uf0b7 1 cup coriander leaves, \n\uf0b7 finely chopped \n\uf0b7 1 cup mint leaves, finely chopped \n\uf0b7 2 or 3 green chilies finely chopped \n\uf0b7 1 tsp. chat masala \n For Frying: \n\uf0b7  1/4 cup besan \n\uf0b7  2 eggs \n\uf0b7  1/4 tsp salt \n\uf0b7  1/4 tsp turmeric powder \n \nMethod \nCook onions in oil until light brown. Add yogurt, garlic & ginger paste, salt, red chili, \ngaram masala, Soya sauce & vinegar to the onions and cook for 6 to 8 minutes. \n Add mince meat t

In [6]:
# SPLIT THE DOCUMENTS INTO CHUNK
txt_spliter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=150,length_function=len)
splits = txt_spliter.split_documents(documents)
print(f"split the documents into {len(splits)} chunks")

split the documents into 134 chunks


In [7]:
#Create embedding
model_name = 'BAAI/bge-small-en'
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}

In [8]:
HF_embedding = HuggingFaceBgeEmbeddings(model_name=model_name, model_kwargs=model_kwargs,
                                      encode_kwargs=encode_kwargs)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/90.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/684 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [9]:
#Vector store 
vectorstore = FAISS.from_documents(documents=splits, embedding=HF_embedding)
retriever = vectorstore.as_retriever()

In [10]:
def docs2str(documents):
    return "\n\n".join(doc.page_content for doc in documents)

In [11]:
# RAG
template = """Answer the following question based on this context:
{context}
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

llm = ChatGroq(temperature=0)

final_rag_chain = ({"context": retriever | docs2str, "question": RunnablePassthrough()} 
                   | prompt
                   | llm
                   | StrOutputParser()
)

In [12]:
question = "how to cook Haleem?"
response = final_rag_chain.invoke(question)
print(f"Question: {question}")
print(f"Answer: {response}")


Question: how to cook Haleem?
Answer: To cook Haleem, follow these steps:

1. Heat remaining oil in a pan and stir-fry onions until they are golden brown. Keep them aside in the pan until required. Just before serving, heat the onions again in the pan.
2. Add boneless beef to the pan and stir-fry until it changes color.
3. Add all the other ingredients except ladyfinger, salt, and chopped coriander. Stir well and cook on low heat for 2-3 hours, or until the beef is tender and the mixture is thick.
4. Add ladyfinger, salt, and stir well. Cook for another 5-10 minutes.
5. Pour the mixture into a serving dish.
6. Garnish with chopped coriander.
7. Serve hot and crisp with loli, rice, or parathas.


**---------------------------------------------------------------------------The more advance RAG System---------------------------------------------------------------------------------**

In [13]:
#
contextualize_q_system_prompt = """
Given a chat history and the latest user question
which might reference context in the chat history,
formulate a standalone question which can be understood
without the chat history. Do NOT answer the question,
just reformulate it if needed and otherwise return it as is.
"""

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

contextualize_chain = contextualize_q_prompt | llm | StrOutputParser()
print(contextualize_chain.invoke({"input": "How to cook Haleem?", "chat_history": []}))

You're asking for instructions on how to prepare Haleem. Here's the standalone question: "What are the steps to cook Haleem?"


In [14]:
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

qa_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI assistant. Use the following context to answer the user's question."),
    ("system", "Context: {context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}")
])

question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [16]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = []
question1 = "Ingradient for cooking Haleem?"
answer1 = rag_chain.invoke({"input": question1, "chat_history": chat_history})['answer']
chat_history.extend([
    HumanMessage(content=question1),
    AIMessage(content=answer1)
])

print(f"Human: {question1}")
print(f"AI: {answer1}\n")

question2 = "how to cook Haleem?"
answer2 = rag_chain.invoke({"input": question2, "chat_history": chat_history})['answer']
chat_history.extend([
    HumanMessage(content=question2),
    AIMessage(content=answer2)
])

print(f"Human: {question2}")
print(f"AI: {answer2}")


Human: Ingradient for cooking Haleem?
AI: The ingredients required for cooking Haleem are:
1 kg boneless beef, fried onions, raw onions, garlic paste, kheema, chopped coriander, and chopped green chilies.

Human: how to cook Haleem?
AI: To cook Haleem, follow these steps:

1. Heat some oil in a pan and fry the onions until they are golden brown. Keep them aside in the pan for later use.
2. In the same pan, add the boneless beef and cook it until it is browned.
3. Add the fried onions, raw onions, garlic paste, kheema, chopped coriander, and chopped green chilies to the pan. Mix well and cook for a few minutes.
4. Add water to the pan and bring the mixture to a boil. Reduce the heat to low, cover the pan, and simmer for several hours until the beef is tender and the flavors are well combined.
5. Just before serving, heat the onions in the pan and add all the seasoning. Stir and cook for a few seconds.
6. Add ladyfinger and salt, stir and pour into a serving dish.
7. Garnish with chopped

**-------------------------Multi user chatbot using SQ-lite-----------------------------**

In [19]:
#settig for SQ lite data base system
import sqlite3
from datetime import datetime
import uuid

DB_NAME = "rag_app.db"

def get_db_connection():
    conn = sqlite3.connect(DB_NAME)
    conn.row_factory = sqlite3.Row
    return conn

def create_application_logs():
    conn = get_db_connection()
    conn.execute('''CREATE TABLE IF NOT EXISTS application_logs
    (id INTEGER PRIMARY KEY AUTOINCREMENT,
    session_id TEXT,
    user_query TEXT,
    gpt_response TEXT,
    model TEXT,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP)''')
    conn.close()

def insert_application_logs(session_id, user_query, gpt_response, model):
    conn = get_db_connection()
    conn.execute('INSERT INTO application_logs (session_id, user_query, gpt_response, model) VALUES (?, ?, ?, ?)',
                 (session_id, user_query, gpt_response, model))
    conn.commit()
    conn.close()

def get_chat_history(session_id):
    conn = get_db_connection()
    cursor = conn.cursor()
    cursor.execute('SELECT user_query, gpt_response FROM application_logs WHERE session_id = ? ORDER BY created_at', (session_id,))
    messages = []
    for row in cursor.fetchall():
        messages.extend([
            {"role": "human", "content": row['user_query']},
            {"role": "ai", "content": row['gpt_response']}
        ])
    conn.close()
    return messages

# Initialize the database
create_application_logs()


In [21]:
# Example usage for a new user
session_id = str(uuid.uuid4())
question = "How to cook haleem?"
chat_history = get_chat_history(session_id)
answer = rag_chain.invoke({"input": question, "chat_history": chat_history})['answer']
insert_application_logs(session_id, question, answer, "gpt-3.5-turbo")
print(f"Human: {question}")
print(f"AI: {answer}\n")

# Example of a follow-up question
question2 = "ingradient for cooking Haleem?"
chat_history = get_chat_history(session_id)
answer2 = rag_chain.invoke({"input": question2, "chat_history": chat_history})['answer']
insert_application_logs(session_id, question2, answer2, "gpt-3.5-turbo")
print(f"Human: {question2}")
print(f"AI: {answer2}")


Human: How to cook haleem?
AI: To cook Haleem, you need to start by cooking the boneless beef. Heat some oil in a pan, stir-fry onions until they are golden brown and keep them aside. In the same pan, cook the remaining ingredients for Haleem including the browned onions, stir well and check the seasoning. Just before serving, heat everything together, add ladyfinger and salt, stir and pour into a serving dish. Garnish with chopped coriander and serve hot with loli, rice or parathas.

Human: ingradient for cooking Haleem?
AI: The ingredients for cooking Haleem are:

* 1 kg boneless beef
* Onions
* Garlic paste
* Kheema (minced meat)
* Chopped coriander
* Chopped green chilies
* Water
* Salt

Note: The full recipe for Haleem is provided in the context, but the ingredient list above includes only the ingredients directly used in the cooking process.
